In [ ]:
# 아래 import를 위해 선행 설치할 친구들
# !pip install cx_Oracle matplotlib numpy Flask Flask-SocketIO flask-cors eventlet pydub wavio

In [1]:
import os

from flask import flash
# 파일 이름 중복 문제 해결하는 방법
import time
# 파일 업로드시 경로 공격 방지를 막아주는 함수를 제공하는 라이브러리
from werkzeug.utils import secure_filename

# HTTP 요청에서 전송된 이미지 바이트 스트림을 메모리상에서 처리할 때 사용될 수 있습니다.
import io
from io import BytesIO

from flask import Flask, send_file, render_template, request, jsonify, send_from_directory
from flask import session
from flask import redirect
from flask import url_for

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 소켓 사용시 -> 실시간 정보 이동
from flask_socketio import SocketIO, emit

# 다른 도메인에어 오는 요청을 허용하도록 설정하는 라이브러리
from flask_cors import CORS

# 현시점 날짜, 시간정보 알아내는 라이브러리
from datetime import datetime

# 오라클DB랑 연동
import cx_Oracle

# 임시파일과 디렉토리를 만들고 조작하는데 사용되는 라이브러리
import tempfile

# 컨텍스트 매니저를 만들어주고 효율적이고 간결한 코드 작성 가능
from contextlib import contextmanager

import sys

# flask와 함께 사용되는 웹 서버 라이브러리, 비동기 I/O 지원 -> 빠르고 확장석 있는 웹 어플 개발 가능하답니다.
import eventlet
import eventlet.wsgi

# 문자열 64 인코딩/디코딩용 python내장모델
import base64

# 이미지 detect를 위한 라이브러리
import torch

# flask에서 비동기적으로 명령을 수행하게 해주는 함수를 포함한 라이브러리
import subprocess

# Ajax를 통해 사진 데이터가 넘어올 때 사용할 라이브러리
from PIL import Image

# Sysdate를 사용할 수 있게 하는 로직
from datetime import datetime

from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import random

# 없어도 되지만 일단 넣어보는 로직
!git config --global user.email "chj0253@gmail.com"
!git config --global user.name "BusUserYM"

In [ ]:
%pwd

In [ ]:
# 함수만 담아둔 칸

# 파일 이름이 허용된 확장자를 가지고 있는지 여부를 판단하는 함수
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# 사진 분석하는 로직
def send_pic(filename):
    # 로직 변경의 상황이 있을 수 있습니다.
    # --weights => best.pt의 위치를 넣어주시고 (고정된 값으로)
    # detect.py => 사진이 저장될 위치와 txt가 저장된 위치를 변경하면 됩니다 ㅎㅎ^^;; (save_dir 및 save_txt부분)
    # --source 부분을 filename으로 지정하면 될듯? 일단 사진 저장되는 위치를 파악하고 그 부분을 넣어야합니다.
    image_path = UPLOAD_FOLDER
    # command = f'!python yolov7/detect.py --weights C:\\Users\\smhrd\\Yolov7_Detect\\yolov7\\runs\\train\\try3_now_best\\weights/best.pt --img-size 640 --conf 0.25 --source {image_path}/{filename} --save-txt'
    command = [
        'python', 'yolov7/detect.py',
        '--weights', 'C:/Users/smhrd/Yolov7_Detect/yolov7/runs/train/try3_now_best/weights/best.pt',
        '--img-size', '640',
        '--conf', '0.25',
        '--source', f'{image_path}/{filename}',
        '--save-txt',
        '--save-conf'
    ]
    # 시간이 오래 걸리니까 비동기적으로 명령을 실행하게 해주는
    subprocess.run(command)
    
# 데이터를 처리하는 로직입니다.
def process_data(data):
    
    # txt_string 문자열을 \n 기준으로 분리하여 리스트로 저장
    data = data.split('\n')

    # 리스트를 데이터프레임으로 변환
    df = pd.DataFrame(data)

    # 인덱스 열 추가
    df.index = df.index + 1
    
    # 인덱스 열에 따른 변수 삽입
    # 숫자형태로 라벨링 된 df에서 class_name을 지칭하는 부분만 가져오기
    num_dict = {}
    for i in range(len(df)):
        num_dict[f'Num{i}'] = df.iloc[i,0][:1]
    
    # 변수의 class number (0~7)에 따라 class명 한국어로 변경
    # 다중 객체 인식할 때 이런식으로 하면 됩니다.
    cName_lst = []
    for Num in num_dict:
        if num_dict[Num] == '0':
            cName_lst.append('콩밥')
        elif num_dict[Num] == '1':
            cName_lst.append('그릇')
        elif num_dict[Num] == '2':
            cName_lst.append('닭갈비')
        elif num_dict[Num] == '3':
            cName_lst.append('낙지볶음')
        elif num_dict[Num] == '4':
            cName_lst.append('돼지고기볶음')
        elif num_dict[Num] == '5':
            cName_lst.append('배추김치')
        elif num_dict[Num] == '6':
            cName_lst.append('00000000')
        elif num_dict[Num] == '7':
            cName_lst.append('Unknown')

    # cName_lst에 ''인 컬럼이 있을 경우 삭제하는 로직
    cName_lst = [cName for cName in cName_lst if cName != '']
    cName_lst = [cName for cName in cName_lst if cName != '그릇']
    print(cName_lst)
        
    return cName_lst

# df만들기 위해 기본적으로 파일열기
def open_excel():
    # 위치는 알아서
    df = pd.read_excel(r'C:/Users/smhrd/food_info_.xlsx')
    
    # 필요없는 부분 -> Food_group, Food_mf 등등 삭제
    # 쓸 부분 - seq, name, recipe, weigth, calor, crb, protein, fat 씁니다.
    # 컬럼 제거
    df.drop(
        ['식품군','제조사명','지역명','당류(g)','나트륨(mg)','콜레스트롤(mg)'
        ,'포화지방산(g)','트랜스지방(g)'],
        axis=1,
        errors='ignore',
        inplace=True
    )
    
    # Food_name 전처리하기
    df['음식명'] = df['음식명'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))
    df.head()
    
    # 중복 확인
    df.iloc[:,:7][df.duplicated(subset=['FOOD_SEQ','음식명'])]
    
    # 레시피가 nan값이 아닌 애들로만 구성.
    df = df[df['레시피'].notna()]
    # 인덱스 값 새롭게 지정하기
    df = df.reset_index(drop=True)
    return df


# 개인 정보에 기반한 활동 대사량, 기초대사량 구하기
# Physical Activity Level (PAL)
# *args로 하면 가변인자를 사용할 수 있다고 하네요!!
# def user_PAL(*args):  
#     weight, height, age, gender, activity, purpose = args
def user_PAL(weight, height, age, gender, activity, purpose):
    # 기초대사량 -> 키, 몸무게, 나이로 구성된 기본식
    base = 10 * weight + 6.25 * height - 5 * age

    # 성별에 따른 계산
    if gender == 'm':
        man_base = base + 5
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': man_base, 'crb': crb, 'protein': protein, 'fat': fat}
    else:
        woman_base = base - 161
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': woman_base, 'crb': crb, 'protein': protein, 'fat': fat}

    # Eat-Fit 목적에 따른 계산
    if purpose == 'plus':
        user['cal'] *= 1.2
    elif purpose == 'lose':
        user['cal'] *= 0.8

    # 활동량에 따른 계산
    activity_levels = {1: 1.2, 2: 1.375, 3: 1.55, 4: 1.725, 5: 1.9}
    user['cal'] *= activity_levels[activity]
    user['cal'] = int(user['cal'])
    
    # 단위를 맞춰주기 위해 계산 결과를 조정
    user['crb'] = int(round(user['cal'] * crb / 4, 1))
    user['protein'] = int(round(user['cal'] * protein / 4,1))
    user['fat'] = int(round(user['cal'] * fat / 9,1))

    return user

# Ajax를 통해서 넘어온 유저 정보 기반 cal, crb, protein, fat 담아두는 기능.
def user(mem_id, cal, crb, protein, fat):
    user = {'mem_id':mem_id,'cal':cal, 'crb':crb,'protein':protein,'fat':fat}
    return user


# 밥과 면 종류에서 특정 값 내에 부합한 음식만 가져오기
def rice_noodle_rec(df,need_crb, need_cal, need_protein, need_fat):
    food_crb_lst = []
    mem_remain_crb_lst = []
    for i in range(len(df)):
        food_crb=df.iloc[i]['탄수화물(g)']
        food_cal=df.iloc[i]['열량(kcal)']
        food_protein=df.iloc[i]['단백질(g)']
        food_fat=df.iloc[i]['지방(g)']

        remain_user_crb = need_crb
        remain_user_cal = need_cal
        remain_user_protein = need_protein
        remain_user_fat = need_fat

        if (need_crb*0.60) <= food_crb <= (need_crb*0.85):
            food_crb_lst.append(df.iloc[i])
            remain_crb = int(round(remain_user_crb - food_crb,1))
            remain_cal = int(round(remain_user_cal - food_cal,1))
            remain_protein = int(round(remain_user_protein - food_protein,1))
            remain_fat = int(round(remain_user_fat - food_fat,1))

            mem_remain_crb_lst.append({'FOOD_RICE_SEQ':df.iloc[i]['FOOD_SEQ'] ,'remain_crb':remain_crb,
                                   'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
    #print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
    food_crb_df = pd.DataFrame(food_crb_lst)
    mem_remain_crb_df = pd.DataFrame(mem_remain_crb_lst)
    
    # cal 마이너스인거 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_cal']<0:
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
    
    # crb 마이너스 5 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_crb']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
            
    # fat 마이너스 5 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_fat']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
            
     # protein 마이너스 10 이상 삭제하기
    i = 0
    while i < len(mem_remain_crb_df):
        if mem_remain_crb_df.iloc[i]['remain_protein']< (-5):
            mem_remain_crb_df = mem_remain_crb_df.drop(mem_remain_crb_df.index[i])
        else:
            i += 1
    return mem_remain_crb_df

# 국 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_crb_df 고 df2 = soup_df입니다.
def soup_rec(df,df2, need_cal):
    food_soup_lst = []
    mem_remain_cal_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (need_cal*0.1) <= food_cal <= (need_cal*0.2):
                food_soup_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_remain_cal_lst.append({'FOOD_SOUP_SEQ':df2.iloc[i]['FOOD_SEQ'],'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,'remain_crb':remain_crb,
                                           'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

    #print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
    food_soup_df = pd.DataFrame(food_soup_lst)
    mem_remain_cal_df = pd.DataFrame(mem_remain_cal_lst)

    # cal, crb, fat, protein이 각각 -5 미만인 행 삭제
    mem_remain_cal_df = mem_remain_cal_df[(mem_remain_cal_df['remain_cal'] >= -5) & 
                                           (mem_remain_cal_df['remain_crb'] >= -5) & 
                                           (mem_remain_cal_df['remain_fat'] >= -5) & 
                                           (mem_remain_cal_df['remain_protein'] >= -15)]

    # 인덱스 재설정
    mem_remain_cal_df.reset_index(drop=True, inplace=True)
            
    return mem_remain_cal_df

# 메인 반찬 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_cal_df 고 df2 = main_df입니다.
def main_rec(df,df2):
    food_main_lst = []
    mem_remain_fat_protein_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (remain_user_protein*0.65) <= food_protein <= (remain_user_protein*0.85) or (remain_user_fat*0.65)<= food_fat <= (remain_user_fat*0.85):
                food_main_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_remain_fat_protein_lst.append({'FOOD_MAIN_SEQ': df2.iloc[i]['FOOD_SEQ'],'FOOD_SOUP_SEQ':df.iloc[j]['FOOD_SOUP_SEQ'],'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,
                                                   'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

    food_main_df = pd.DataFrame(food_main_lst)
    mem_remain_fat_protein_df = pd.DataFrame(mem_remain_fat_protein_lst)

    # cal, crb, fat, protein이 각각 -5 미만인 행 삭제
    mem_remain_fat_protein_df = mem_remain_fat_protein_df[(mem_remain_fat_protein_df['remain_cal'] >= -5) & 
                                                       (mem_remain_fat_protein_df['remain_crb'] >= -5) & 
                                                       (mem_remain_fat_protein_df['remain_fat'] >= -5) & 
                                                       (mem_remain_fat_protein_df['remain_protein'] >= -15)]

    # 인덱스 재설정
    mem_remain_fat_protein_df.reset_index(drop=True, inplace=True)
            
    return mem_remain_fat_protein_df

# 서브 반찬 종류에서 특정 값 내에 부합한 음식만 가져오기
# df = mem_re_fat_pro_df 고 df2 = sub_df입니다.
def sub_rec(df,df2):
    food_sub_lst =[]
    mem_final_lst = []
    for i in range(len(df2)):
        food_crb=df2.iloc[i]['탄수화물(g)']
        food_cal=df2.iloc[i]['열량(kcal)']
        food_protein=df2.iloc[i]['단백질(g)']
        food_fat=df2.iloc[i]['지방(g)']
        for j in range(len(df)):
            remain_user_crb = df.iloc[j]['remain_crb']
            remain_user_cal = df.iloc[j]['remain_cal']
            remain_user_protein = df.iloc[j]['remain_protein']
            remain_user_fat = df.iloc[j]['remain_fat']

            if (remain_user_protein*0.70) <= food_protein <= remain_user_protein or (remain_user_fat*0.70)<= food_fat <= remain_user_fat or (remain_user_cal*0.7) <= food_cal <= remain_user_cal or (remain_user_crb*0.70) <= food_crb <= remain_user_crb:
                food_sub_lst.append(df2.iloc[i])
                remain_crb = int(round(remain_user_crb - food_crb,1))
                remain_cal = int(round(remain_user_cal - food_cal,1))
                remain_protein = int(round(remain_user_protein - food_protein,1))
                remain_fat = int(round(remain_user_fat - food_fat,1))
                mem_final_lst.append({'FOOD_RICE_SEQ':df.iloc[j]['FOOD_RICE_SEQ'] ,'FOOD_SOUP_SEQ':df.iloc[j]['FOOD_SOUP_SEQ'],'FOOD_MAIN_SEQ': df.iloc[j]['FOOD_MAIN_SEQ'],
                                      'FOOD_SUB_SEQ':df2.iloc[i]['FOOD_SEQ'],'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
    food_sub_df = pd.DataFrame(food_sub_lst)
    mem_final_df = pd.DataFrame(mem_final_lst)

    # cal, crb, fat, protein이 각각 -5 미만인 행 삭제
    mem_final_df = mem_final_df[(mem_final_df['remain_cal']<=50) & (mem_final_df['remain_cal'] >= -5) & (mem_final_df['remain_crb']<=10) & (mem_final_df['remain_crb'] >= -5) & (mem_final_df['remain_fat']<=10) & (mem_final_df['remain_fat'] >= -5) & (mem_final_df['remain_protein']<=5) & (mem_final_df['remain_protein'] >= -15)]

    # 인덱스 재설정
    mem_final_df.reset_index(drop=True, inplace=True)
            
    return mem_final_df

# 식단 추천 로직 만들기
# 얘는 아침밥 먹을 때 사용할 로직
def recommend_m_meal():
    
    user=user_PAL(72,176,25,'m',3,'maintain')
    
    df=open_excel()
    
    # 전체 df에서 한식, 일식만 가져옴
    kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
    # 한,일식 df에서 밥,면 종류만 가져옴
    rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
    soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
    main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
    sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
    
    # 필요 영양성분 데이터
    need_crb=user['crb']/3
    need_cal = user['cal']/3
    need_protein=user['protein']/3
    need_fat = user['fat']/3

    mem_remain_crb_df = rice_noodle_rec(rice_noodle_df, need_crb, need_cal, need_protein, need_fat)
    
    mem_remain_cal_df = soup_rec(mem_remain_crb_df, soup_df, need_cal)
    
    mem_remain_fat_protein_df =main_rec(mem_remain_cal_df, main_df)
    
    mem_final_df =sub_rec(mem_remain_fat_protein_df, sub_df)
    
    return mem_final_df


In [ ]:
app = Flask(__name__)
# flask 어플리케이션에서 세션을 사용하려면 반드시 있어야하는 'SECRET_KEY'라고 합니다. 없으면 세션 사용 불가능!
app.config['SECRET_KEY'] = 'today is good day but weather is sooooo bad that make me sad'
CORS(app)

# 이런 식으로 쓸수 있습니다.
@app.route('/')
def index():
    return '실행이 잘 되고 있는가...확인 중!'


# YOLOv7 경로 설정
os.environ["PATH"] += os.pathsep + "C:\\Users\\smhrd\\Yolov7_Detect\\yolov7\\runs\\train\\try3_now_best\\weights\\best.pt"


# flask를 통해 업로드된 파일을 저장한 디렉토리 경로
################################################################################################
# images , EatFit2 프로젝트 명 이 2가지만 변경하면 됩니다
################################################################################################
UPLOAD_FOLDER = r'C:/eGovFrame-4.0.0/workspace.edu/EatFit2/src/main/webapp/resources/food1/'


# 아래의 파일 형태만 업로드 가능하다 라는 의미.
ALLOWED_EXTENSIONS = {'jpg','jpeg','JPG','JPEG','png','PNG','GIF','gif'}
# flask에서 아래에 업로드된 값을 'app.config'를 통해 전역 환경 변수로 사용할 수 있게 만드는 로직
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# 이게 맞네요.
@app.route('/upload-image', methods=['POST'])
def upload():
    image_data = request.files['image']
    new_image_data = image_data.filename[:-4]
    image_data.save(UPLOAD_FOLDER,f'{new_image_data}.png')
    filename = f'{new_image_data}.png'
    
    return redirect(url_for('analyze', filename=filename))


# 사진 분석 로직
@app.route('/analyze', methods = ['GET'])           # 분석 로직 다 적어야 합니다.
def analyze():
    filename = request.args.get('filename')
    print(filename)
    img = Image.open(UPLOAD_FOLDER,f'{filename}')
    # 사진 분석
    send_pic(filename)
    
    return redirect(url_for('send_class', filename=filename))



@app.route('/send_class/<filename>', methods=['GET'])
def send_class(filename):
    txt_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/labels'
    file_name = filename[:-4]
    with open(f'{txt_path}/{file_name}.txt', 'r') as f:
        data = f.read()
        
    processed_data = process_data(data)
    
    return redirect(url_for('send_image', filename=filename, cnames=','.join(processed_data)))




# 분석된 이미지를 화면에 보여주는 로직 => 성공
@app.route('/send_image', methods=['GET'])
# 여기서 매개변수에 filename을 넣어도 괜찮습니다.
def send_image():
    # 이미지 분석 파일 위치임다.
    filename = request.args.get('filename')
    cnames = request.args.get('cnames')
    if cnames:
        cnames = cnames.split(',')
        print(cnames)
    image_path = r'C:/eGovFrame-4.0.0/workspace.edu/EatFit2/src/main/webapp/resources/food2/'
    origin_image_path = UPLOAD_FOLDER
    # 이건 항상 변할 수 있습니다. png를 넣으면 pgn가 나옴
    img = 'snd'+f'{filename}'
    image = os.path.join(image_path,img)
    
    analyze_path = image
    origin_path = os.path.join(origin_image_path,img)

    return jsonify({
        'analyze_image':img,
        'origin_image':filename,
        'cNames':cnames
    })

# Ajax를 통해서 정보를 전달 받아서 (user의 cal,crb,protein,fat) 기반으로
# 맞는 음식 찾기.
# user_info => json형태로 받아올 이름.
@app.route('/get_user_data', methods=['POST'])
def rec_meal():
    
    user= request.get_json()
    
    mem_id = user["mem_id"]
    need_cal = user["cal"]
    need_crb = user["crb"]
    need_protein = user["protein"]
    need_fat = user["fat"]

    df = open_excel()
     
    # 전체 df에서 한식, 일식만 가져옴
    kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
    # 한,일식 df에서 밥,면 종류만 가져옴
    rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
    soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
    main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
    sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
    
    mem_remain_crb_df = rice_noodle_rec(rice_noodle_df, need_crb, need_cal, need_protein, need_fat)
    
    mem_remain_cal_df = soup_rec(mem_remain_crb_df, soup_df, need_cal)
    
    mem_remain_fat_protein_df =main_rec(mem_remain_cal_df, main_df)
    
    mem_final_df =sub_rec(mem_remain_fat_protein_df, sub_df)
    
    return redirect(url_for('send_rec_meal', mem_id=mem_id))
mem_final_df # mem_id 보내기


# ajax를 통한 전송용 로직
@app.route('/send_rec_meal', methods=['GET'])
def ajax_data():
    df = rec_meal()
    
    df_len = len(df)
    rand_num = random.randint(0, df_len-1)
    
    rec_dic={}
    
    rec_dic ={
        'mem_id': mem_id,
        'rice_seq' : df.iloc[rand_num]['FOOD_RICE_SEQ'],
        'soup_seq' : df.iloc[rand_num]['FOOD_SOUP_SEQ'],        
        'main_seq' :df.iloc[rand_num]['FOOD_MAIN_SEQ'],        
        'sub_seq' : df.iloc[rand_num]['FOOD_SUB_SEQ'],
        'remain_crb' : df.iloc[rand_num]['remain_crb'],                
        'remain_cal' : df.iloc[rand_num]['remain_cal'],                
        'remain_protein' : df.iloc[rand_num]['remain_protein'],
        'remain_fat' : df.iloc[rand_num]['remain_fat']
        }
    
    return jsonify(rec_dic)

# 플라스크 open.
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
# app = Flask(__name__)
# # flask 어플리케이션에서 세션을 사용하려면 반드시 있어야하는 'SECRET_KEY'라고 합니다. 없으면 세션 사용 불가능!
# app.config['SECRET_KEY'] = 'today is good day but weather is sooooo bad that make me sad'
# CORS(app)

# # 이런 식으로 쓸수 있습니다.
# @app.route('/')
# def index():
#     return '실행이 잘 되고 있는가...확인 중!'

# # YOLOv7 경로 설정
# os.environ["PATH"] += os.pathsep + "C:\\Users\\smhrd\\Yolov7_Detect\\yolov7\\runs\\train\\try3_now_best\\weights\\best.pt"


# # flask를 통해 업로드된 파일을 저장한 디렉토리 경로
# ################################################################################################
# # images , EatFit2 프로젝트 명 이 2가지만 변경하면 됩니다
# ################################################################################################
# UPLOAD_FOLDER = r'C:/eGovFrame-4.0.0/workspace.edu/EatFit2/src/main/webapp/resources/images'
# # 아래의 파일 형태만 업로드 가능하다 라는 의미.
# ALLOWED_EXTENSIONS = {'jpg','jpeg','JPG','JPEG','png','PNG','GIF','gif'}
# # flask에서 아래에 업로드된 값을 'app.config'를 통해 전역 환경 변수로 사용할 수 있게 만드는 로직
# app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# # 이게 맞네요.
# @app.route('/upload-image', methods=['POST'])
# def upload():
#     image_data = request.files['image']
#     new_image_data = image_data.filename[:-4]
#     image_data.save(f'uploads/{new_image_data}.png')
#     filename = f'{new_image_data}.png'
    
#     return redirect(url_for('analyze', filename=filename))


# # 사진 분석 로직
# @app.route('/analyze', methods = ['GET'])           # 분석 로직 다 적어야 합니다.
# def analyze():
#     filename = request.args.get('filename')
#     print(filename)
#     img = Image.open(f'uploads/{filename}')
#     # 사진 분석
#     send_pic(filename)
    
#     return redirect(url_for('send_class', filename=filename))


# @app.route('/send_class/<filename>', methods=['GET'])
# def send_class(filename):
#     txt_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/labels'
#     file_name = filename[:-4]
#     with open(f'{txt_path}/{file_name}.txt', 'r') as f:
#         data = f.read()
        
#     processed_data = process_data(data)
    
#     return redirect(url_for('send_image', filename=filename, cnames=','.join(processed_data)))


# # 분석된 이미지를 화면에 보여주는 로직 => 성공
# @app.route('/send_image', methods=['GET'])
# # 여기서 매개변수에 filename을 넣어도 괜찮습니다.
# def send_image():
#     # 이미지 분석 파일 위치임다.
#     filename = request.args.get('filename')
#     cnames = request.args.get('cnames')
#     if cnames:
#         cnames = cnames.split(',')
#         print(cnames)
#     image_path = r'C:/Users/smhrd/Yolov7_Detect/img_results/'
#     origin_image_path = r'C:/Users/smhrd/Yolov7_Detect/uploads/'
#     # 이건 항상 변할 수 있습니다. png를 넣으면 pgn가 나옴
#     img = f'{filename}'
#     image = os.path.join(image_path,img)

#     analyze_path = image
#     origin_path = os.path.join(origin_image_path,img)
    
    
#     # 이미지를 로드하고 base64로 인코딩합니다.
#     #def load_and_encode_image(path):
#        # with open(path, "rb") as image_file:
#       #      encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
#      #   return encoded_image
#    # encoded_analyze_image = load_and_encode_image(analyze_path)
#   #  encoded_origin_image = load_and_encode_image(origin_path)
    

#     return jsonify({
#         'analyze_image':analyze_path,
#         'origin_image':origin_path,
#         'cNames':cnames
#     })


# # Ajax를 통해서 정보를 전달 받아서 (user의 cal,crb,protein,fat) 기반으로
# # 맞는 음식 찾기.
# # user_info => json형태로 받아올 이름.
# @app.route('/get_user_data', methods=['POST'])
# def rec_meal():
    
#     user= request.get_json()
    
#     mem_id = user["mem_id"]
#     need_cal = user["cal"]
#     need_crb = user["crb"]
#     need_protein = user["protein"]
#     need_fat = user["fat"]

#     df = open_excel()
     
#     # 전체 df에서 한식, 일식만 가져옴
#     kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
#     # 한,일식 df에서 밥,면 종류만 가져옴
#     rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
#     soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
#     main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
#     sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
    
#     mem_remain_crb_df = rice_noodle_rec(rice_noodle_df, need_crb, need_cal, need_protein, need_fat)
    
#     mem_remain_cal_df = soup_rec(mem_remain_crb_df, soup_df, need_cal)
    
#     mem_remain_fat_protein_df =main_rec(mem_remain_cal_df, main_df)
    
#     mem_final_df =sub_rec(mem_remain_fat_protein_df, sub_df)
    
#     return redirect(url_for('send_rec_meal', mem_id=mem_id))
# mem_final_df # mem_id 보내기


# # ajax를 통한 전송용 로직
# @app.route('/send_rec_meal', methods=['GET'])
# def ajax_data():
#     df = rec_meal()
    
#     df_len = len(df)
#     rand_num = random.randint(0, df_len-1)
    
#     rec_dic={}
    
#     rec_dic ={
#         'mem_id': mem_id,
#         'rice_seq' : df.iloc[rand_num]['FOOD_RICE_SEQ'],
#         'soup_seq' : df.iloc[rand_num]['FOOD_SOUP_SEQ'],        
#         'main_seq' :df.iloc[rand_num]['FOOD_MAIN_SEQ'],        
#         'sub_seq' : df.iloc[rand_num]['FOOD_SUB_SEQ'],
#         'remain_crb' : df.iloc[rand_num]['remain_crb'],                
#         'remain_cal' : df.iloc[rand_num]['remain_cal'],                
#         'remain_protein' : df.iloc[rand_num]['remain_protein'],
#         'remain_fat' : df.iloc[rand_num]['remain_fat']
#         }
    
#     return jsonify(rec_dic)



# # 데이터베이스 연결
# def conn_database():
#     conn = cx_Oracle.connect(
#         user='foodhero',
#         password='12345',
#         dsn=cx_Oracle.makedsn('project-db-stu.ddns.net', 1524, service_name='xe')
#     )
#     return conn
           
# #데이터베이스 연결 테스트
# def test_connection():
#     try:
#         conn = conn_database()
#         print("OracleDB 연결 성공")
#     except cx_Oracle.DatabaseError as e:
#         print("OracleDB 연결 실패:", e)
#     finally:
#         if 'conn' in locals():
#             conn.close()
# test_connection()

# # 분석된 사진 및 이전 사진 등 사용자 정보를 저장하는 로직
# # def save_db_data(mem_id, food_seq, food_img, analyze_img):
# #     with conn.cursor() as cursor:
# #         cursor.execute("""
# #             INSERT INTO MEMBER_DR (MEM_id, req_date, MLD, Food_seq, Food_img, Food_weight, analyze_img)
# #             VALUES (:mem_id, NULL, NULL, :food_seq, :food_img, NULL, :analyze_img)
# #         """, {'mem_id': mem_id,
# #               'food_seq': food_seq,
# #               'food_img': food_img,
# #               'analyze_img': analyze_img})
# #         conn.commit()

# # 플라스크 open.
# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)